In [1]:
import pandas as pd

In [2]:
def merge_geo_coordinates(df):
    geo = pd.read_csv(f"../../data/geocodings.csv")[["address", "lat", "long"]]
    
    # Copying these two aren't expensive
    return_geo = geo.rename(columns={"address": "Return station",
                                     "lat": "Return lat",
                                     "long": "Return long"})
    depart_geo = geo.rename(columns={"address": "Departure station",
                                     "lat": "Departure lat",
                                     "long": "Departure long"})
    
    tmp_df = pd.merge(df, return_geo, on="Return station", how="inner")
    res_df = pd.merge(tmp_df, depart_geo, on="Departure station", how="inner")
    
    return res_df.dropna()

In [3]:
def get_useful_columns(df):
    new_df = df[["Departure", "Return", "Departure station", "Return station"]]
    return new_df

In [4]:
def process_df():
    dfs = []
    
    for y in range(2017, 2022):
        tmp_df = pd.read_csv(f"../../data/Mobi_System_data_{y}.csv", compression='zip')
        dfs.append(get_useful_columns(tmp_df))
    
    df = pd.concat(dfs)
    df["Departure"] = pd.to_datetime(df["Departure"], errors="coerce")
    df["Return"] = pd.to_datetime(df["Return"], errors="coerce")
    df.dropna(inplace=True)

    df["DepartureYear"] = df["Departure"].dt.year
    df["DepartureMonth"] = df["Departure"].dt.month
    df["ReturnYear"] = df["Return"].dt.year
    df["ReturnMonth"] = df["Return"].dt.month
    df.drop(columns=["Departure", "Return"], inplace=True)
    
    return df

In [5]:
df = process_df()

In [6]:
def get_routes(df, top_n, month, year):
    df = df[(df["DepartureMonth"] == month) & (df["ReturnMonth"] == month)]
    df = df[(df["DepartureYear"] == year) & (df["ReturnYear"] == year)]
    
    df.drop(columns=["DepartureMonth", "ReturnMonth", "DepartureYear", "ReturnYear"], inplace=True)
    df = merge_geo_coordinates(df)
    
    df = df.groupby(["Departure station", "Return station", "Return lat", "Return long", "Departure lat", "Departure long"])\
           .size()\
           .reset_index(name="Count")
    df = df.sort_values(by="Count", ascending=False).set_index("Count")
    df = df.head(top_n)
    return df.to_dict()

In [7]:
get_routes(df, 5, 3, 2017)

{'Departure station': {109: '0035 Coal Harbour Community Centre',
  82: '0028 Davie & Beach',
  62: '0187 Bute & Davie',
  58: '0050 Bute & Comox'},
 'Return station': {109: '0035 Coal Harbour Community Centre',
  82: '0028 Davie & Beach',
  62: '0036 Bute & Robson',
  58: '0032 Comox & Denman'},
 'Return lat': {109: 49.290419,
  82: 49.28752799999999,
  62: 49.28590800000001,
  58: 49.288444},
 'Return long': {109: -123.12581,
  82: -123.142139,
  62: -123.1271193,
  58: -123.139203},
 'Departure lat': {109: 49.290419,
  82: 49.28752799999999,
  62: 49.28222400000001,
  58: 49.283343},
 'Departure long': {109: -123.12581,
  82: -123.142139,
  62: -123.132778,
  58: -123.1310991}}